# Candidate Informaton:
## Name: Tahawar ihsan
## Email: tahawarihsan@gmail.com



In [47]:
#installation required
! pip install langchain
! pip install unstructured
! pip install pdf2image
! pip install pdfminer
! pip install pdfminer.six
! pip install markdown
! pip install openai
! pip install reportlab
! pip install pymupdf
! pip install -U langchain-community



In [48]:
# pip install unstructured_inference
# !pip install -U langchain-community

# Source Material:
URL: https://drive.google.com/uc?export=download&id=1WYaEoSozvUmJ_Lt5clAVs8NuSu43h9ps

# Data Loading

In [ ]:
from langchain_community.document_loaders import OnlinePDFLoader
import re

In [ ]:
# loader = OnlinePDFLoader("https://drive.google.com/file/d/1WYaEoSozvUmJ_Lt5clAVs8NuSu43h9ps/view")

In [50]:
from langchain_community.document_loaders import PyMuPDFLoader
import re
loader = PyMuPDFLoader("/content/crime-and-punishment (3).pdf")

In [51]:
data = loader.load()
data[0]

Document(page_content='Download free eBooks of classic literature, books and \nnovels at Planet eBook. Subscribe to our free eBooks blog \nand email newsletter.\nCrime and Punishment\nBy Fyodor Dostoevsky\n', metadata={'source': '/content/crime-and-punishment (3).pdf', 'file_path': '/content/crime-and-punishment (3).pdf', 'page': 0, 'total_pages': 767, 'format': 'PDF 1.5', 'title': 'Crime and Punishment', 'author': 'Fyodor Dostoevsky', 'subject': 'Download classic literature as completely free eBooks from Planet eBook. ', 'keywords': '', 'creator': 'Adobe InDesign CS2 (4.0)', 'producer': 'Adobe PDF Library 7.0', 'creationDate': "D:20080206205229+11'00'", 'modDate': "D:20080706190506+10'00'", 'trapped': ''})

In [52]:
data[0].dict().keys()

dict_keys(['page_content', 'metadata', 'type'])

In [53]:
data[0].type

'Document'

In [54]:
data[0].page_content

'Download free eBooks of classic literature, books and \nnovels at Planet eBook. Subscribe to our free eBooks blog \nand email newsletter.\nCrime and Punishment\nBy Fyodor Dostoevsky\n'

In [55]:
data[1].dict()["page_content"][:1000]

'Crime and Punishment\n\uf645\nTranslator’s Preface\nA \nfew words about Dostoevsky himself may help the Eng-\nlish reader to understand his work.\nDostoevsky was the son of a doctor. His parents were \nvery hard- working and deeply religious people, but so poor \nthat they lived with their five children in only two rooms. \nThe father and mother spent their evenings in reading aloud \nto their children, generally from books of a serious charac-\nter.\nThough always sickly and delicate Dostoevsky came out \nthird in the final examination of the Petersburg school of \nEngineering. There he had already begun his first work, \n‘Poor Folk.’\nThis story was published by the poet Nekrassov in his \nreview and was received with acclamations. The shy, un-\nknown youth found himself instantly something of a \ncelebrity. A brilliant and successful career seemed to open \nbefore him, but those hopes were soon dashed. In 1849 he \nwas arrested.\nThough neither by temperament nor conviction a revol

In [56]:
data[0].metadata



{'source': '/content/crime-and-punishment (3).pdf',
 'file_path': '/content/crime-and-punishment (3).pdf',
 'page': 0,
 'total_pages': 767,
 'format': 'PDF 1.5',
 'title': 'Crime and Punishment',
 'author': 'Fyodor Dostoevsky',
 'subject': 'Download classic literature as completely free eBooks from Planet eBook. ',
 'keywords': '',
 'creator': 'Adobe InDesign CS2 (4.0)',
 'producer': 'Adobe PDF Library 7.0',
 'creationDate': "D:20080206205229+11'00'",
 'modDate': "D:20080706190506+10'00'",
 'trapped': ''}

In [57]:
# Assuming `data` is a list of Document objects loaded from the PyMuPDFLoader
total_characters = sum(len(page.page_content) for page in data)
full_text = " ".join([page.page_content for page in data])

print(f"Total characters in the PDF: {total_characters}")


Total characters in the PDF: 1170703


# Data Preprocessing

In [58]:
import re

def preprocess_text(text):
    # More targeted removal of unwanted patterns, ensure headings are not removed
    patterns_to_remove = [
        r'\n\d+\x18\x18\n',  # sequence of digit followed by non-printable characters
        r'\nFree eBooks at Planet eBook.com\n',
        r'\n\x18{2,3}\d?\n?',  # non-printable characters that might not be necessary
    ]

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text)

    # Simplify multiple newlines to a single newline everywhere except around potential headings
    text = re.sub(r'\n+(?!Part |Chapter )', '\n', text)

    return text


In [59]:
def detect_part_boundaries(text):
    part_pattern = r'Part [IVXLCDM]+\n'
    part_headings = re.findall(part_pattern, text)
    parts = []

    if not part_headings:
        print("No part headings found. Ensure text is preprocessed correctly.")
        return parts

    # Handle extraction of parts with detected headings
    part_positions = [text.find(heading) for heading in part_headings] + [len(text)]
    for i in range(len(part_headings)):
        part_text = text[part_positions[i]:part_positions[i + 1]]
        parts.append((part_headings[i].strip(), part_text))

    return parts


In [60]:
def detect_chapter_boundaries(part_text, part_heading):
    chapter_pattern = r'Chapter [IVXLCDM]+\n'
    chapters = re.split(chapter_pattern, part_text)

    # Remove the first split if it's just a repeat of the part heading or very short (likely not a real chapter)
    if chapters and (chapters[0].strip() == part_heading.strip() or len(chapters[0].strip()) < 50):
        chapters = chapters[1:]

    return [ch.strip() for ch in chapters if ch.strip()]


In [61]:
# First, you need to preprocess the entire text.
# full_text = " ".join([page.page_content for page in data])  # Assuming 'data' is a list of document pages.
cleaned_text = preprocess_text(full_text)

# Now, call detect_part_boundaries with the cleaned text.
parts = detect_part_boundaries(cleaned_text)




In [62]:
# Check if parts have been detected and handle them accordingly.
if parts:
    print(f"Detected {len(parts)} parts.")
    # Now you can process these parts or access a specific part like so:
    part1 = parts[0]  # Access the first part, which is a tuple of (part heading, part text)
else:
    print("No parts were detected.")

Detected 6 parts.


# Formatting and Saving in correct format

In [63]:
import os
def create_markdown_file(parts):
    output_dir = 'markdown_output'
    os.makedirs(output_dir, exist_ok=True)

    for part_num, (part_heading, part_text) in enumerate(parts):
        part_number = part_num + 1
        part_filename = os.path.join(output_dir, f'part_{part_number}_{part_heading.strip()}.md')

        with open(part_filename, 'w', encoding='utf-8') as part_file:
            # Write the part heading
            part_file.write(f"# {part_heading.strip()}\n\n")

            # Detect and write chapters within the part
            chapters = detect_chapter_boundaries(part_text, part_heading)
            for chapter_num, chapter_text in enumerate(chapters, 1):
                part_file.write(f"## Chapter {chapter_num}\n\n{chapter_text}\n\n")

            print(f"Part {part_number} written to {part_filename}")


In [64]:
# Load and preprocess your text
full_text = " ".join(page.page_content for page in data)
cleaned_text = preprocess_text(full_text)

# Detect parts and handle them
parts = detect_part_boundaries(cleaned_text)
create_markdown_file(parts)


Part 1 written to markdown_output/part_1_Part I.md
Part 2 written to markdown_output/part_2_Part II.md
Part 3 written to markdown_output/part_3_Part III.md
Part 4 written to markdown_output/part_4_Part IV.md
Part 5 written to markdown_output/part_5_Part V.md
Part 6 written to markdown_output/part_6_Part VI.md


# Splitting

In [68]:
import os
import glob
from langchain.text_splitter import MarkdownHeaderTextSplitter

# Set up the directory containing the Markdown files
markdown_directory = 'markdown_output'  # Assuming you store Markdown files in this directory

headers_to_split_on = [
    ("#", "Header 1"),   # Split at main parts
    ("##", "Header 2"), # Split at chapters within parts
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

splits = []

for markdown_file in glob.glob(os.path.join(markdown_directory, "*.md")):
    # Read the Markdown file
    file_name = os.path.basename(markdown_file).replace("_", ' ').split('.')[0]
    print(f"Processing file: {file_name}")

    with open(markdown_file, 'r', encoding='utf-8') as file:
        markdown_text = file.read()

    # Split the Markdown text based on headers
    md_header_splits = markdown_splitter.split_text(markdown_text)

    # Collecting splits
    for split in md_header_splits:
        splits.append(split)

print(f"Total Documents after split: {len(splits)}")


Processing file: part 4 Part IV
Processing file: part 1 Part I
Processing file: part 6 Part VI
Processing file: part 3 Part III
Processing file: part 2 Part II
Processing file: part 5 Part V
Total Documents after split: 39


In [69]:
print("Total Documents: {}".format(len(splits)))

Total Documents: 39


In [70]:
splits[10]

Document(page_content='‘O\nf course, I’ve been meaning lately to go to Razumi-\nhin’s to ask for work, to ask him to get me lessons or\nsomething …’ Raskolnikov thought, ‘but what help can\nhe be to me now? Suppose he gets me lessons, suppose he\nshares his last farthing with me, if he has any farthings, so\nthat I could get some boots and make myself tidy enough to\ngive lessons … hm … Well and what then? What shall I do\nwith the few coppers I earn? That’s not what I want now. It’s\nreally absurd for me to go to Razumihin….’\nThe question why he was now going to Razumihin agi-\ntated him even more than he was himself aware; he kept\nuneasily seeking for some sinister significance in this ap-\nparently ordinary action.\n‘Could I have expected to set it all straight and to find a\nway out by means of Razumihin alone?’ he asked himself\nin perplexity.\nHe pondered and rubbed his forehead, and, strange to\nsay, after long musing, suddenly, as if it were spontaneously\nand by chance, a fa

In [71]:
len(splits[0].dict()["page_content"])

27411

In [72]:
splits[0].dict()["metadata"]

{'Header 1': 'Part IV', 'Header 2': 'Chapter 1'}

# OpenAI API Setup

In [73]:
import configparser
import openai

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the config.ini file
config.read('config.ini')

my_openai_key = config['openai']['OPENAI_API_KEY']
openai.api_key = my_openai_key
os.environ['OPENAI_API_KEY'] = my_openai_key

# Prompt, LLM, Chain, Callback

In [74]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI


# handler = StdOutCallbackHandler()


# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

In [75]:
with get_openai_callback() as cb:
    stuff_chain.run([md_header_splits[0]])
    print(cb)

Tokens Used: 9185
	Prompt Tokens: 8928
	Completion Tokens: 257
Successful Requests: 1
Total Cost (USD): $0.048495


In [76]:
summarize_document = {}

def insert_summary(part_no, chapter_no, summary):
    if part_no not in summarize_document:
        summarize_document[part_no] = {}
    summarize_document[part_no][chapter_no] = summary

count = 1
cost = 0

for split_doc in splits:
    with get_openai_callback() as cb:
        print(count)
        print(split_doc.dict()["metadata"]['Header 1'])
        print(split_doc.dict()["metadata"]['Header 2'])
        partNo = split_doc.dict()["metadata"]['Header 1']
        chapterNo = split_doc.dict()["metadata"]['Header 2']
        summary = stuff_chain.run([split_doc])
        print(cb)
        cost += cb.total_cost

        insert_summary(partNo, chapterNo, summary)

    count += 1
#     break

1
Part IV
Chapter 1
Tokens Used: 7436
	Prompt Tokens: 7250
	Completion Tokens: 186
Successful Requests: 1
Total Cost (USD): $0.03904
2
Part IV
Chapter 2
Tokens Used: 6970
	Prompt Tokens: 6805
	Completion Tokens: 165
Successful Requests: 1
Total Cost (USD): $0.0365
3
Part IV
Chapter 3
Tokens Used: 4573
	Prompt Tokens: 4426
	Completion Tokens: 147
Successful Requests: 1
Total Cost (USD): $0.024335
4
Part IV
Chapter 4
Tokens Used: 8931
	Prompt Tokens: 8787
	Completion Tokens: 144
Successful Requests: 1
Total Cost (USD): $0.046095000000000004
5
Part IV
Chapter 5
Tokens Used: 10306
	Prompt Tokens: 10109
	Completion Tokens: 197
Successful Requests: 1
Total Cost (USD): $0.0535
6
Part IV
Chapter 6
Tokens Used: 4264
	Prompt Tokens: 4101
	Completion Tokens: 163
Successful Requests: 1
Total Cost (USD): $0.022949999999999998
7
Part I
Chapter 1
Tokens Used: 4764
	Prompt Tokens: 4574
	Completion Tokens: 190
Successful Requests: 1
Total Cost (USD): $0.02572
8
Part I
Chapter 2
Tokens Used: 10475
	Prom

In [77]:
with get_openai_callback() as cb:
        print(splits[-1].dict()["metadata"]['Header 1'])
        partNo = splits[-1].dict()["metadata"]['Header 1']
        summary = stuff_chain.run([splits[-1]])
        print(cb)
        cost += cb.total_cost

        insert_summary(partNo, " ", summary)

Part V
Tokens Used: 7683
	Prompt Tokens: 7549
	Completion Tokens: 134
Successful Requests: 1
Total Cost (USD): $0.039755


In [78]:
print(f"Total Cost (USD): ${cost}")

Total Cost (USD): $1.6073549999999999


# Summarized Document (Dictionary)

In [42]:
summarize_document

{'Part IV': {'Chapter 1': "Raskolnikov is visited by Svidrigaïlov, who expresses a desire to make his acquaintance and seek his assistance regarding Raskolnikov's sister, Avdotya Romanovna. Svidrigaïlov, who has a dubious past, including the suspicious death of his wife Marfa Petrovna, proposes to give Avdotya ten thousand roubles to help her break off her engagement with Mr. Luzhin, whom Svidrigaïlov deems unsuitable. Raskolnikov is skeptical and hostile, suspecting ulterior motives. Svidrigaïlov also shares eerie experiences of seeing his deceased wife’s ghost, which Raskolnikov dismisses as madness. Despite Raskolnikov's refusal to help, Svidrigaïlov insists on his intentions and reveals that Marfa Petrovna left Avdotya three thousand roubles in her will. The conversation ends with Svidrigaïlov leaving, hinting at further intentions to see Avdotya.",
  'Chapter 2': 'In this passage from "Crime and Punishment," Raskolnikov and Razumihin hurry to Bakaleyev’s to meet Luzhin. On the way

In [43]:
import json
# Specify the file path where you want to save the JSON data
json_file_path = "summarize_document.json"

# Convert the dictionary to JSON and save it to the file
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(summarize_document, json_file, ensure_ascii=False, indent=4)

# Print a message indicating that the data has been saved
print(f"JSON data saved to {json_file_path}")

JSON data saved to summarize_document.json


In [44]:
# Roman numeral to integer conversion function
def roman_to_int(roman):
    roman_numerals = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
    result = 0
    i = 0
    while i < len(roman):
        if i + 1 < len(roman) and roman_numerals[roman[i]] < roman_numerals[roman[i + 1]]:
            result -= roman_numerals[roman[i]]
        else:
            result += roman_numerals[roman[i]]
        i += 1
    return result

# Check if "Translator’s Preface" exists and reorder the dictionary
# summarize_document = {
#     "Translator’s Preface": "Preface text...",
#     "Part IV": "Text for Part IV...",
#     "Part I": "Text for Part I..."
# }


    # If there is no "Translator’s Preface", sort normally
reordered_data = {
        key: summarize_document[key] for key in sorted(summarize_document.keys(), key=lambda x: roman_to_int(x.split()[-1]) if x.startswith("Part") else 1000)
    }

# Specify the file path where you want to save the JSON data
json_file_path = "summarize_documentor.json"

# Convert the reordered dictionary to JSON and save it to the file
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(reordered_data, json_file, ensure_ascii=False, indent=4)

# Print a message indicating that the data has been saved
print(f"JSON data saved to {json_file_path}")


JSON data saved to summarize_documentor.json


In [45]:
reordered_data

{'Part I': {'Chapter 1': 'On a sweltering July evening, a young man named Raskolnikov leaves his cramped garret in S. Place and heads towards K. bridge, carefully avoiding his landlady due to his overwhelming debt and fear of confrontation. He is deeply absorbed in his own thoughts, struggling with poverty and a sense of isolation that has led him to a state of near-hypochondria. As he walks through the stifling streets of Petersburg, he becomes increasingly agitated and introspective, contemplating a vague, daring plan. He arrives at a large, dilapidated house and visits an old pawnbroker, Alyona Ivanovna, to pawn a watch. The interaction is tense and leaves Raskolnikov feeling disgusted and conflicted. After leaving, he is overcome with loathing for his thoughts and actions, and in a state of distress, he enters a tavern to drink beer, hoping to clear his mind. The tavern is nearly empty, and as he drinks, he momentarily feels a sense of relief, though he remains aware that his impro

# PDF Generation of Summary

In [46]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

# Create a PDF document
pdf_file_path = "summarize_document.pdf"
doc = SimpleDocTemplate(pdf_file_path, pagesize=letter)

story = []

# Define padding values
padding_left = 50  # Adjust the left padding
padding_right = 50  # Adjust the right padding
padding_top = 50  # Adjust the top padding
padding_bottom = 50  # Adjust the bottom padding

# Define a style for the title
title_style = getSampleStyleSheet()["Title"]
title_style.fontName = "Helvetica-Bold"
title_style.alignment = 1  # Centered
title_style.fontSize = 42  # Increase the font size for the title
title_style.leading = 56  # Increase the line spacing (leading) for the title

# Define a style for the top-level section titles (Heading1)
styles = getSampleStyleSheet()
heading1_style = styles["Heading1"]
heading1_style.fontSize = 36  # Increase the font size for Heading1
heading1_style.leading = 48  # Increase the line spacing (leading) for the title

heading2_style = styles["Heading2"]
heading2_style.fontSize = 28  # Increase the font size for Heading1
heading2_style.leading = 36  # Increase the line spacing (leading) for the title

# Define a style for the paragraphs
style = styles["Normal"]
style.fontSize = 14  # Increase the font size for the paragraphs
style.leading = 20  # Increase the line spacing (leading) for the title

# Add the title to the PDF
title_text = "Summary: Crime and Punishment"
title = Paragraph(title_text, title_style)
story.append(title)

# Add spacing above the title
story.append(Spacer(1, padding_top))

# Iterate through the JSON data and add the "Translator’s Preface," parts, chapters, and summaries to the PDF
for section, chapters in reordered_data.items():
    # Add the section title to the PDF
    section_title = Paragraph(section, heading1_style)  # Use the Heading1 style for top-level titles
    story.append(section_title)

    for chapter, summary in chapters.items():
        # Skip the empty chapter title for the "Translator’s Preface"
        if chapter:
            # Add the chapter title to the PDF
            chapter_title = Paragraph(chapter, heading2_style)
            story.append(chapter_title)

        # Add the chapter summary to the PDF
        chapter_summary = Paragraph(summary, style)
        story.append(chapter_summary)

        # Add space between chapters
    story.append(Spacer(1, padding_bottom + 20))  # Add spacing below each chapter

# Build the PDF document
doc.build(story)

print(f"PDF created: {pdf_file_path}")


PDF created: summarize_document.pdf
